In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


In [2]:
sentence="Repeat is the best medicine for memory".split()
vocab = list(set(sentence))
print(vocab)

['for', 'Repeat', 'is', 'memory', 'medicine', 'best', 'the']


In [4]:
word2index={tkn:i for i, tkn in enumerate(vocab,1)}
word2index['<unk>']=0
print(word2index)

{'for': 1, 'Repeat': 2, 'is': 3, 'memory': 4, 'medicine': 5, 'best': 6, 'the': 7, '<unk>': 0}


In [5]:
print(word2index['best'])

6


In [7]:
index2word={v:k for k,v in word2index.items()}
print(index2word)
print(index2word[3])

{1: 'for', 2: 'Repeat', 3: 'is', 4: 'memory', 5: 'medicine', 6: 'best', 7: 'the', 0: '<unk>'}
is


In [13]:
def build_data(sentence, word2index):
    encoded=[word2index[token] for token in sentence]
    #print(encoded)
    input_seq, label_seq=encoded[:-1], encoded[1:]
    #print(input_seq)
    #print(label_seq)
    input_seq=torch.LongTensor(input_seq).unsqueeze(0) # 차원추가
    label_seq=torch.LongTensor(label_seq).unsqueeze(0)
    return input_seq, label_seq


In [14]:
X, Y=build_data(sentence, word2index)

In [15]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size):
        super(Net, self).__init__()
        self.embedding_layer=nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=input_size)
        self.rnn_layer=nn.RNN(input_size, hidden_size, batch_first=True)
        self.linear=nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output=self.embedding_layer(x)
        # 2. RNN층
        # 크기변화 : (배치크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치크기, 은닉층 크기)
        output, hidden=self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치크기, 시퀀스 길이, 은닉층 크기) => (배치크기,  시퀀스 길이, 단어장 크기)
        output=self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치크기, 시퀀스 길이, 단어장 크기) => (배치크기 * 시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [16]:
vocab_size=len(word2index)
input_size=5
hidden_size=20


In [17]:
model=Net(vocab_size, input_size, hidden_size)
loss_function=nn.CrossEntropyLoss()
optimizer=optim.Adam(params=model.parameters())


In [18]:
output=model(X)
print(output)

tensor([[ 0.1537,  0.1138, -0.2179,  0.1028, -0.4076,  0.1756, -0.2627, -0.4279],
        [ 0.0250,  0.2747,  0.2019,  0.0611, -0.1623,  0.0186, -0.1487,  0.1581],
        [ 0.1002,  0.0852, -0.2719,  0.0546, -0.0565, -0.0424, -0.2199,  0.0091],
        [ 0.0577, -0.1286,  0.1325,  0.1659, -0.2342,  0.0269, -0.0606, -0.0092],
        [-0.1093, -0.2124, -0.0093,  0.0852,  0.1044, -0.2753, -0.0377,  0.0371],
        [-0.1092, -0.1863,  0.1633,  0.1252, -0.0256, -0.2056, -0.0078, -0.1126]],
       grad_fn=<ViewBackward0>)


In [20]:
decded=lambda y: [index2word.get(x) for x in y]
print(decded)


<function <lambda> at 0x00000295EC0565F0>


In [24]:
for step in range(201):
    optimizer.zero_grad()
    outputs=model(X)
    loss=loss_function(outputs, Y.view(-1))

    loss.backward()
    optimizer.step()

    if step % 40 == 0:
        print("step:",step, "loss:",loss.item())
        pred=outputs.softmax(-1).argmax(-1).tolist()
        print(" ".join(['Repeat']+decded(pred)))
        print()


step: 0 loss: 0.03506462275981903
Repeat is the best medicine for memory

step: 40 loss: 0.029277389869093895
Repeat is the best medicine for memory

step: 80 loss: 0.024855181574821472
Repeat is the best medicine for memory

step: 120 loss: 0.021390261128544807
Repeat is the best medicine for memory

step: 160 loss: 0.01861913688480854
Repeat is the best medicine for memory

step: 200 loss: 0.01636502705514431
Repeat is the best medicine for memory

